# Chapter 02

## Class Definitions
In python, every data is an instance of some class.


In [13]:
print(list().__class__.__name__)
print(str().__class__.__name__)
print(dict().__class__.__name__)

list
str
dict


In [22]:
print(list().__class__.__class__.__name__)

type


The `self` identifier explicitly identifies the instance that a method is invoked in class.


In [24]:
class CreditCard:
    """A consumer credit card"""
    
    def __init__(self, customer, bank, acnt, limit):
        """Create a new credit card instance.
        
        The initial balance is zero.
        
        :param customer: the name of the customer 
        :param bank: the name of the bank
        :param acnt: the account identifier
        :param limit: credit limit
        """
        
        self._customer = customer
        self._bank = bank
        self._account = acnt
        self._limit = limit
        self._balance = 0
        
    def get_customer(self):
        """Return name of the customer"""
        return self._customer
    
    def get_bank(self):
        """Return the bank's name."""
        return self._bank
    
    def get_account(self):
        """Return the card identifying number (typically stored as a string."""
        return self._account
    
    def get_limit(self):
        """Return current credit limit."""
        return self._limit
    
    def get_balance(self):
        """Return current balance."""
        return self._balance
    
    def charge(self, price):
        """Charge given price to the card, assuming sufficient credit limit.
        
        Return True if charge was processed; False if charge was denied.
        """
        
        if price + self._balance > self._limit:
            return False
        else:
            self._balance += price
            return True
    
    def make_payment(self, amount):
        """Process customer payment that reduces balance."""
        self._balance -= amount 

Python interpreter automatically binds the instance upon which the method is invoked to the `self` parameter.


In [27]:
cc = CreditCard('John Doe', 'Bank', '1234 5678', 1000)
cc

In class, `__init__` method works as the **constructor** of the class. Also, single leading underscore in the name of a data member, such as `_balance`, implies that it is intended as **nonpublic**. Users of a class should not directly access such members.

For better encapsulation, it is mostly better to treat all data members as nonpublic and provide accessors, to provide a user of our class read-only access to a trait, and update methods for updating its memebers.

### Operator overloading and Python's Special Methods
By default, operators can not work on classes unless special methods are defined. You can see detailed info at [here](https://docs.python.org/3/reference/datamodel.html#special-method-names). Python also supports non-operator overloads. For example, `str` invokes `__str__()` and `bool` invokes `__bool__()`. However, we should not carelessly assume that Python will manage all the implications. Defining `__eq__` will support syntax `a == b`, but it does not affect the evaluation of a syntax `a != b`, which should be defined by `__ne__`. In similar context, defining `__eq__` and `__lt__` does not imply semantics for a `a <= b `.

